In [1]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/schiend/Desktop/DOHERTY/R-workspace/cecelia/inst')

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [7]:
base_dir = '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/Zum0pK/')
#im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
im_path = os.path.join(zero_dir, 'ccidCorrected.zarr')
#im_path = os.path.join(zero_dir, 'ccidSlidingWindow.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 'Zum0pK')

In [3]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()
napari_utils.task_dir = task_dir

Could not find Java JRE compatible with x86_64 architecture
napari.manifest -> 'skan' could not be imported: Could not find file 'napari.yaml' in module 'skan'


In [4]:
channel_names = [
    'EB', 'T-A', 'T-B', 'AF', 'gen'
]

In [8]:
napari_utils.open_image(
    im_path,
    use_channel_axis = True, as_dask = True,
    channel_names = channel_names,
    downsample_z = True
)

[60, 4, 14, 525, 526]


In [ ]:
# save image as tiff
import py.tiff_utils as tiff_utils

tiff_utils.save_as_tiff(
    '/Volumes/USER_data/Dominik/Experiments/TEAGEN_SKIN/DATA/PuJo5p.tif',
    napari_utils.im_data[0], napari_utils.dim_utils, imagej = True)

In [ ]:
print('here')

In [ ]:
napari_utils.show_labels_all(
    show_labels = False,
    show_points = False,
    show_tracks = True,
    value_names = ['tcells.gBT']
)

In [ ]:
from napari_animation import AnimationWidget
animation_widget = AnimationWidget(napari_utils.viewer)
napari_utils.viewer.window.add_dock_widget(animation_widget, area='right')

In [9]:
# show populations
napari_utils.show_pop_mapping('live')

(live) tcells.cd8/tracked
(live) neutro/tracked
(live) tracked/contact.live.tcells.cd8__tracked


In [10]:
pop_type = 'live'
pop_map = napari_utils.pop_utils.pop_map(napari_utils.task_dir, pop_type)
pop_data = napari_utils.pop_utils.pop_data(napari_utils.task_dir, pop_type)

In [16]:
pop_info = pop_map['5iaWSo']

In [17]:
pop_info

{'name': ['contact.live.tcells.cd8__tracked'],
 'colour': ['#00ffff'],
 'show': [True],
 'parent': ['neutro/tracked'],
 'path': ['neutro/tracked/contact.live.tcells.cd8__tracked'],
 'valueName': ['tcells.cd8'],
 'filterValues': [True],
 'filterMeasure': ['live.cell.contact#live.tcells.cd8/tracked'],
 'filterFun': ['eq'],
 'filterDefaultAll': [False],
 'isTrack': [False]}